In [4]:
import pandas as pd
from selenium import webdriver
import time as t
from bs4 import BeautifulSoup


In [5]:

def import_ibv():
    # definindo a url da b3
    url = "https://sistemaswebb3-listados.b3.com.br/indexPage/day/IBOV?language=pt-br"

    # iniciando o webdriver, que deve estar com o driver na pasta do arquivo para funcionar, nesse caso será utilizado do Chrome
    browser = webdriver.Chrome()

    # Entrando na url informada
    browser.get(url)

    # Encontrando o elemento no html para filtrarmos os dados que precisamos
    browser.find_element_by_id('segment').send_keys('Setor de Atuação')

    # Aguardando 2 segundos para a página atualizar
    t.sleep(2)

    # Agora selecionando o o menu para podermos ver todas as ações disponíveis
    browser.find_element_by_id('selectPage').send_keys('120')

    # Aguardando 1 segundo para o página atualizar
    t.sleep(1)

    # IMportando o html inteiro para o Python na variável html
    html = browser.page_source

    # Fechando o navegador
    browser.close()

    # Usando o beatiful soup e convertendo o arquivo para podermos selecionar os dados que precisamos
    bs = BeautifulSoup(html, 'html.parser')

    # Criando as listas onde iremos armazenar os dados do html
    setor, cod, acao, tipo, qtde_t, part, part_a = [], [], [], [], [], [], []

    # Usando a função find_all para selecionarmos apenas 'td' que contém os dados que usaremos e armazenando eles na variável linhas
    linhas = bs.find_all('td')

    # Criando uma variável que será usada no while loop abaixo
    i = 0

    # While loop para podermos armazenar os dados em suas respectivas colunas, onde:
    # temos 7 colunas então pegamos o primeiro elemento 0 e somando 1 até chegarmos ao último elemento para a coluna [i+6] = ao setimo elemento
    # por fim somando 7 para pularmos para a próxima linhas e seguir a lógica
    # o loop só irá para quando o número de linhas for maior do que o que os dados apresentados, para isso usados o lógica len(linhas) que nos entrega a quantia de dados
    # e como i é somado de 7 a cada iteração o loop só irá para quando todos os dados forem indexidaodos nas suas devidas colunas
    while i < (len(linhas)):
        setor.append(linhas[i].text)
        cod.append(linhas[i+1].text)
        acao.append(linhas[i+2].text)
        tipo.append(linhas[i+3].text)
        qtde_t.append(linhas[i+4].text)
        part.append(linhas[i+5].text)
        part_a.append(linhas[i+6].text)
        i += 7

    # Criando um df para armazearmos os dados
    df = pd.DataFrame({'Setor': setor[:-1],
                       'Acao': cod[:-1],
                       'Empresa': acao[:-1],
                       'Qntd_teorica': qtde_t[:-1],
                       'Part_%': part[:-1]})  # filtramos para eliminar as últimas linhas, pois a b3 entrega uma soma ao final que não queremos
    # Separando os setores e subsetores em colunas
    df['SubSetor'] = df['Setor'].apply(lambda s: s[s.rfind('/')+1:].strip())
    df['Setor'] = df['Setor'].apply(lambda s: s[:s.rfind('/')])

    # Convertendo os valores de string para int
    df['Qntd_teorica'] = df['Qntd_teorica'].apply(lambda s: s.replace(".", ""))
    df['Qntd_teorica'] = pd.to_numeric(df['Qntd_teorica'])
    df['Part_%'] = df['Part_%'].apply(lambda s: s.replace(",", ""))
    df['Part_%'] = pd.to_numeric(df['Part_%'])/1000
    df.sort_values('Part_%', ascending=False, inplace=True)

    # Criando uma colunas com a % acumulada
    df['Part_%_acum'] = df['Part_%'].cumsum()

    # Resetando o index
    df.reset_index(drop=True, inplace=True)

    # fim
    return df




In [6]:

df = import_ibv()

print(df)


                             Setor   Acao       Empresa  Qntd_teorica  Part_%  \
0                    Mats Básicos   VALE3          VALE    3843570705  16.292   
1   Petróleo, Gás e Biocombustívei  PETR4     PETROBRAS    4566442248   6.572   
2                 Financ e Outros   ITUB4  ITAUUNIBANCO    4780002924   5.793   
3                 Financ e Outros   BBDC4      BRADESCO    4691427537   4.627   
4   Petróleo, Gás e Biocombustívei  PETR3     PETROBRAS    2703343822   4.324   
..                             ...    ...           ...           ...     ...   
86                      Bens Indls  ECOR3   ECORODOVIAS     339237914   0.118   
87                 Consumo Cíclico  JHSF3     JHSF PART     305915142   0.087   
88                 Consumo Cíclico  EZTC3         EZTEC     101618236   0.078   
89                  Tec.Informação  CASH3        MELIUZ     548153725   0.052   
90                  Comput e Equip  POSI3  POSITIVO TEC      78053723   0.030   

                           